In [1]:
import torch

# Torch pipeline
Use a torch model as encoding model to predict brain data.
First let's define an experiment name and some parameters.

In [2]:
experiment = 'torch_alexnet'
batch_size = 300
# Pick mps for Apple chip, cuda for nvidia & cpu otherwise
device = 'mps'  # @param ['cpu', 'cuda'] {allow-input: true}

In [3]:
device = torch.device(device)

# Configure folders
challenge_data_dir = '../data/algonauts_2023_challenge_data'
exp_output_dir = f'../data/out/{experiment}'

# Get feature extractor
from src.algonauts.feature_extractors.TorchFeatureExtractor import TorchFeatureExtractor
random_seed = 5
feature_extractor = TorchFeatureExtractor(batch_size, random_seed, device)

## Load model
Define a function to load and return the model, printing its layers.
Due to memory constraints, we will delete the model from memory in each loop. Here we first load it, print its
nodes and delete it.

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names


def load_model():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
    model.to(device)  # send the model to the chosen device ('cpu' or 'cuda')
    model.eval()  # set the model to evaluation mode, since you are not training it
    return model


model = load_model()
train_nodes, _ = get_graph_node_names(model)
print(train_nodes)
del model

## Select layers and subjects
Now let's define which layer(s) we will pick from, and which subject(s) to process

In [5]:
layers = ['features.12']
subjects = [1, 2, 3, 4, 5, 6, 7, 8]

## Run pipeline

In [ ]:
from src.algonauts.evaluators import Correlations
from src.algonauts.encoders import LinearEncoder
from src.algonauts.data_processors.NSDDataset import NSDDataset

for layer_name in layers:
    print(f'Running for layer {layer_name}')
    for subj in subjects:
        print(f'Running for subject {subj}')

        # Set data directories based on parameters
        output_dir = f'{exp_output_dir}/{layer_name}'
        dataset = NSDDataset(challenge_data_dir, output_dir, subj)

        # Load model for the iteration
        model = load_model()

        # Extract features from the layer
        train_features, val_features, test_features = feature_extractor.extract_all_features(dataset, model, layer_name)

        # Delete model to free up memory
        del model

        # Apply PCA and fit linear encoder to get fMRI predictions
        lh_fmri_val_pred, rh_fmri_val_pred = LinearEncoder.predict(dataset, train_features, val_features, test_features)
        # Calculate correlations for each hemisphere
        lh_correlation = Correlations.calculate_correlation(lh_fmri_val_pred, dataset.lh_fmri_val)
        rh_correlation = Correlations.calculate_correlation(rh_fmri_val_pred, dataset.rh_fmri_val)
        Correlations.plot_and_write_correlations(dataset, lh_correlation, rh_correlation, exp_output_dir, layer_name, subj)